In [33]:
import datetime
import os

import constants

from azure.ai.ml import command, Input, Output
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Environment
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential



In [34]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=constants.SUBSCRIPTION_ID,
    resource_group_name=constants.RESOURCE_GROUP_NAME,
    workspace_name=constants.WORKSPACE_NAME,
)

In [35]:
%%writefile components/prep.py 

from azure.storage.blob import BlobServiceClient
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
from pathlib import Path
import datetime
import json

credential = DefaultAzureCredential()
# Check if given credential can get token successfully.
credential.get_token("https://management.azure.com/.default")
secret_client = SecretClient(vault_url="https://mlgroup.vault.azure.net/", credential=credential)

import argparse

parser = argparse.ArgumentParser("prep")
parser.add_argument("--blob_storage", type=str, help="Mounted Azure ML blob storage")
parser.add_argument("--prep_output")
args = parser.parse_args()

# log in to the Blob Service Client
blob_storage = args.blob_storage
blob_storage_key = secret_client.get_secret("blob-storage-key")
blob_service_client = BlobServiceClient(blob_storage, account_key=blob_storage_key.value)

# connect to the container 
container_client = blob_service_client.get_container_client(container="stock-news-json") 

# list and download all currently available blobs
blob_list = container_client.list_blobs()
print(f"Blob from: {blob_storage} has these blobs today: {blob_list}")

# get the timestamp with the current day 
current_day_date = datetime.datetime.today().isoformat()[:10]

blobs_to_use = [blob.name for blob in blob_list if current_day_date in blob.name]
for blob in blobs_to_use:
      print(f"Downloading blob: {blob}")
      blob_client = blob_service_client.get_blob_client(container="stock-news-json", blob=blob)
      with open(blob, mode="wb") as sample_blob:
            download_stream = blob_client.download_blob()
            sample_blob.write(download_stream.readall())

all_data_dict = {}
for json_file in blobs_to_use:
      with open(json_file,"r+") as file:
      # First we load existing data into a dict.
            file_data = json.load(file)
            all_data_dict.update(file_data)
            
with open((Path(args.prep_output) / "merged_stock_news.json"), "w") as file:
      file.write(json.dumps(all_data_dict, indent=4))

# this is a comments

Overwriting components/prep.py


In [36]:
%%writefile components/classify.py

from pathlib import Path
import argparse
import json
import os

from transformers import AutoTokenizer, AutoModelForSequenceClassification

parser = argparse.ArgumentParser()
parser.add_argument("--classify_input", type=str, help="Mounted Azure ML blob storage")
parser.add_argument("--classify_output", type=str, help="Mounted Azure ML blob storage")
args = parser.parse_args()

# download distilbert model from HuggingFace
tokenizer = AutoTokenizer.from_pretrained("KernAI/stock-news-destilbert")
model = AutoModelForSequenceClassification.from_pretrained("KernAI/stock-news-destilbert")

# retriev the list of blobs from the current day - input is a .txt file
with open(os.path.join(args.classify_input, "merged_stock_news.json"), "r") as f:
      data = json.load(f)

# get a list of all tickers in the data   
tickers = list(data.keys())
for ticker in tickers:
      texts = data[ticker]["texts"]
      sentiments = []
      for text in texts: 
            tokenized_text = tokenizer(
                  text,
                  truncation=True,
                  is_split_into_words=False,
                  return_tensors="pt"
            )

            outputs = model(tokenized_text["input_ids"])
            outputs_logits = outputs.logits.argmax(1)

            mapping = {0: 'neutral', 1: 'negative', 2: 'positive'}
            predicted_label = mapping[int(outputs_logits[0])]
            sentiments.append(predicted_label)

      # add the sentiments to the data
      data[ticker]["sentiments"] = sentiments

# overwrite old files with new files containing the sentiment
with open((Path(args.classify_output) / "merged_stock_news.json"), "w") as f:
      json.dump(data, f)

Overwriting components/classify.py


In [37]:
%%writefile components/summarize.py

from azure.storage.blob import BlobServiceClient
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient

from pathlib import Path
import datetime 
import argparse
import json
import os

from transformers import PegasusTokenizer, PegasusForConditionalGeneration, TFPegasusForConditionalGeneration

credential = DefaultAzureCredential()
# Check if given credential can get token successfully.
credential.get_token("https://management.azure.com/.default")
secret_client = SecretClient(vault_url="https://mlgroup.vault.azure.net/", credential=credential)

parser = argparse.ArgumentParser()
parser.add_argument("--summarize_input", type=str, help="Mounted Azure ML blob storage")
parser.add_argument("--summarize_output", type=str, help="Mounted Azure ML blob storage")
args = parser.parse_args()

# load the model and the tokenizer
tokenizer = PegasusTokenizer.from_pretrained("human-centered-summarization/financial-summarization-pegasus")
model = PegasusForConditionalGeneration.from_pretrained("human-centered-summarization/financial-summarization-pegasus") 

# retriev the list of blobs from the current day - input is a .txt file
with open(os.path.join(args.summarize_input, "merged_stock_news.json"), "r") as f:
      data = json.load(f)

tickers = list(data.keys())
for ticker in tickers:
      texts = data[ticker]["texts"]

      summaries = []
      for text in texts: 
            # Tokenize our text
            # If you want to run the code in Tensorflow, please remember to return the particular tensors as simply as using return_tensors = 'tf'
            input_ids = tokenizer(text, return_tensors="pt").input_ids

            # Generate the output (Here, we use beam search but you can also use any other strategy you like)
            output = model.generate(
                  input_ids, 
                  max_length=32, 
                  num_beams=5, 
                  early_stopping=True
            )

            # Finally, we can print the generated summary
            summaries.append(tokenizer.decode(output[0], skip_special_tokens=True))

      # add the sentiments to the data
      data[ticker]["summaries"] = summaries
      
data = json.dumps(data)

# connect and authenticate to the blob client
account_url = "https://mlstorageleo.blob.core.windows.net"
file_name = f"processed-stock-news-{datetime.datetime.today().isoformat()[:10]}.json"

# Create the BlobServiceClient object
blob_storage_key = secret_client.get_secret("blob-storage-key")
blob_service_client = BlobServiceClient(account_url, credential=blob_storage_key.value)
blob_client = blob_service_client.get_blob_client(container="processed-stock-news-json", blob=file_name)
blob_client.upload_blob(data)

# overwrite old files with new files containing the sentiment
with open((Path(args.summarize_output) / "merged_stock_news.json"), "w") as f:
      json.dump(data, f)

Overwriting components/summarize.py


In [38]:
%%writefile components/notify.py

from pathlib import Path
import datetime 
import argparse
import json
import os

from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
from azure.communication.email import EmailClient

parser = argparse.ArgumentParser()
parser.add_argument("--notify_input", type=str, help="Mounted Azure ML blob storage")
parser.add_argument("--notify_output", type=str, help="Mounted Azure ML blob storage")
args = parser.parse_args()

credential = DefaultAzureCredential()
credential.get_token("https://management.azure.com/.default")
secret_client = SecretClient(vault_url="https://mlgroup.vault.azure.net/", credential=credential)

conenction_string = secret_client.get_secret("mail-connection-string")
email_client = EmailClient.from_connection_string(conenction_string.value)

with open(os.path.join(args.notify_input, "merged_stock_news.json"), "r") as f:
      data = json.load(f)

msft_summaries = " ".join([i+nl for i in data["MSFT"]["summaries"] if "All photographs subject to copyright." not in i])
msft_sentiments = (data["MSFT"]["sentiments"].count("positive"), data["MSFT"]["sentiments"].count("neutral"), data["MSFT"]["sentiments"].count("negative"))
msft_texts = " ".join([i+nl for i in data["MSFT"]["url"]])

nl = "\n"
text = f"""
This is your daily stock news summary. 

News about Microsoft: 
{msft_summaries}
\n Sentiments: positive -> {msft_sentiments[0]} neutral -> {msft_sentiments[1]}, negative -> {msft_sentiments[2]}

Full texts: 
Microsoft 
{msft_texts}

"""
# News about DigitalOcean:
# {" ".join([i+nl for i in data["DOCN"]["summaries"] if "All photographs subject to copyright." not in i])}
# \n Sentiments: positive -> {data["DOCN"]["sentiments"].count("positive")}, neutral -> {data["DOCN"]["sentiments"].count("neutral")}, negative -> {data["DOCN"]["sentiments"].count("negative")}

# News about Apple:
# {" ".join([i+nl for i in data["AAPL"]["summaries"] if "All photographs subject to copyright." not in i])}
# \n Sentiments: positive -> {data["AAPL"]["sentiments"].count("positive")}, neutral -> {data["AAPL"]["sentiments"].count("neutral")}, negative -> {data["AAPL"]["sentiments"].count("negative")}

# News about Boradcom: 
# {" ".join([i+nl for i in data["AVGO"]["summaries"] if "All photographs subject to copyright." not in i])}
# \n Sentiments: positive -> {data["AVGO"]["sentiments"].count("positive")}, neutral -> {data["AVGO"]["sentiments"].count("neutral")}, negative -> {data["AVGO"]["sentiments"].count("negative")}

# News about IBM:
# {" ".join([i+nl for i in data["IBM"]["summaries"] if "All photographs subject to copyright." not in i])}
# \n Sentiments: positive -> {data["IBM"]["sentiments"].count("positive")}, neutral -> {data["IBM"]["sentiments"].count("neutral")}, negative -> {data["IBM"]["sentiments"].count("negative")}

# News about Texas Instruments:
# {" ".join([i+nl for i in data["TXN"]["summaries"] if "All photographs subject to copyright." not in i])}
# \n Sentiments: positive -> {data["TXN"]["sentiments"].count("positive")}, neutral -> {data["TXN"]["sentiments"].count("neutral")}, negative -> {data["TXN"]["sentiments"].count("negative")}


# DigialOcean
# {" ".join([i+nl for i in data["DOCN"]["url"]])}

# Apple
# {" ".join([i+nl for i in data["AAPL"]["url"]])}

# Broadcom
# {" ".join([i+nl for i in data["AVGO"]["url"]])}

# IBM 
# {" ".join([i+nl for i in data["IBM"]["url"]])}

# Texas Instruments 
# {" ".join([i+nl for i in data["TXN"]["url"]])}

message = {
    "content": {
        "subject": f"Stock news analysis for {datetime.datetime.today().isoformat()[:10]}",
        "plainText": text
    },
    "recipients": {
        "to": [
            {
                "address": "leopuettmann@outlook.de",
                "displayName": "Leo"
            }
        ]
    },
    "senderAddress": "DoNotReply@632a8f5c-5cc8-4c44-8e7e-f509c76d0d24.azurecomm.net"
}

poller = email_client.begin_send(message)
print(poller.result())

# overwrite old files with new files containing the sentiment
with open((Path(args.notify_output) / "merged_stock_news.json"), "w") as f:
      json.dump(data, f)

Overwriting components/notify.py


In [39]:
%%writefile dependencies/conda.yml
name: stock-analysis-env
channels:
  - conda-forge
dependencies:
  - python=3.9
  - pip
  - pip:
    - azure-storage-blob
    - azure-identity
    - azure-keyvault
    - azure-communication-email
    - transformers
    - torch
    - sentencepiece
    - numpy

Overwriting dependencies/conda.yml


In [40]:
custom_env_name = "stock-analysis-env"
version = "1.7"

try:    
    pipeline_job_env = ml_client.environments.get(custom_env_name, version=version)

except:
    pipeline_job_env = Environment(
        name=custom_env_name,
        description="Custom environment for stock analysis pipeline",
        conda_file=os.path.join("dependencies", "conda.yml"),
        image="mcr.microsoft.com/azureml/curated/python-sdk-v2:4",
        version=version,
    )
    pipeline_job_env = ml_client.environments.create_or_update(pipeline_job_env)

    print(
        f"Environment with name {pipeline_job_env.name} is registered to workspace, the environment version is {pipeline_job_env.version}"
    )

In [41]:
data_type = AssetTypes.URI_FOLDER
path = "azureml://datastores/stocknewsjson/stock-news-json"
input_mode = InputOutputModes.RO_MOUNT
output_mode = InputOutputModes.RW_MOUNT

In [42]:
data_prep_component = command(
    name="data_prep",
    display_name="Finding out which blobs to actually use",
    description="Loads files from Azure Blob Storage from todays ",
    inputs={
        "blob_storage": Input(mode=InputOutputModes.DIRECT)
    },
    outputs={
        "prep_output": Output(type=data_type, mode=output_mode)
    },
    code="./components/prep.py",
    command="python prep.py --blob_storage ${{inputs.blob_storage}} --prep_output ${{outputs.prep_output}}",
    environment=f"{pipeline_job_env.name}:{pipeline_job_env.version}",
    compute="ava",
    is_deterministic="false"
)

In [43]:
classify_component = command(
    name="classify",
    display_name="Classify the sentiments of todays stock news",
    description="Loads data via AlphaVantage API input, preps data and stores to as data asset",
    inputs={
        "classify_input": Input(type=data_type, mode=input_mode), 
    },
    outputs={
        "classify_output": Output(type=data_type, mode=output_mode)
    },
    code="./components/classify.py",
    command="python classify.py --classify_input ${{inputs.classify_input}} --classify_output ${{outputs.classify_output}}",
    environment=f"{pipeline_job_env.name}:{pipeline_job_env.version}",
    compute="ava",
    is_deterministic="false"
)

In [44]:
summarize_component = command(
    name="summarize",
    display_name="Summarize the news",
    description="Uses a pegasus model to summarize the news aricle",
    inputs={
        "summarize_input": Input(type=data_type, mode=input_mode),
    },
    outputs={
        "summarize_output": Output(type=data_type, mode=output_mode)
    },
    code="./components/summarize.py",
    command="python summarize.py --summarize_input ${{inputs.summarize_input}} --summarize_output ${{outputs.summarize_output}}",
    environment=f"{pipeline_job_env.name}:{pipeline_job_env.version}",
    compute="ava",
    is_deterministic="false"
)

In [45]:
notify_component = command(
    name="notify",
    display_name="Notify the user via Mail",
    description="Sends out an E-Mail with the results of the pipeline",
    inputs={
        "notify_input": Input(type=data_type, mode=input_mode),
    },
    outputs={
        "notify_output": Output(type=data_type, mode=output_mode)
    },
    code="./components/notify.py",
    command="python notify.py --notify_input ${{inputs.notify_input}} --notify_output ${{outputs.notify_output}}",
    environment=f"{pipeline_job_env.name}:{pipeline_job_env.version}",
    compute="ava",
    is_deterministic="false"
)

In [46]:
from azure.ai.ml.dsl import pipeline

@pipeline(compute="ava")
def stock_news_pipeline():

    data_prep_job = data_prep_component(
        blob_storage="https://mlstorageleo.blob.core.windows.net/"
    )
    classify_job = classify_component(
        classify_input=data_prep_job.outputs.prep_output

    ) # feed putput of previous step into the training job
    summarize_job = summarize_component(
        summarize_input = classify_job.outputs.classify_output
    )

    notify_job = notify_component(
        notify_input = summarize_job.outputs.summarize_output
    )

    return {"processed_file": notify_job.outputs.notify_output}

pipeline_job = stock_news_pipeline()

# set pipeline level compute
pipeline_job.settings.default_compute = "ava"
pipeline_job.settings.reuse_component = "false"

In [47]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="stock-news-analysis-pipeline"
)
pipeline_job

Uploading notify.py (< 1 MB): 100%|##########| 4.01k/4.01k [00:00<00:00, 60.4kB/s]




Experiment,Name,Type,Status,Details Page
stock-news-analysis-pipeline,plucky_door_98shy3xpjz,pipeline,Preparing,Link to Azure Machine Learning studio


In [48]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

RunId: plucky_door_98shy3xpjz
Web View: https://ml.azure.com/runs/plucky_door_98shy3xpjz?wsid=/subscriptions/5a361d37-b562-4eee-981b-0936493063e9/resourcegroups/mlgroup/workspaces/mlworkspace

Streaming logs/azureml/executionlogs.txt

[2023-07-31 21:36:41Z] Submitting 1 runs, first five are: a05356c3:daea695b-bd86-4075-a507-eeececa70459
[2023-07-31 21:37:14Z] Completing processing run id daea695b-bd86-4075-a507-eeececa70459.
[2023-07-31 21:37:14Z] Submitting 1 runs, first five are: bb6fe364:6a70ed82-3505-4e73-91e7-0f246c4ac44b
[2023-07-31 21:38:06Z] Completing processing run id 6a70ed82-3505-4e73-91e7-0f246c4ac44b.
[2023-07-31 21:38:06Z] Submitting 1 runs, first five are: d2aa7c57:a1fc8b31-40be-4734-86c8-c08c8b8b6d10
[2023-07-31 21:53:39Z] Completing processing run id a1fc8b31-40be-4734-86c8-c08c8b8b6d10.
[2023-07-31 21:53:39Z] Submitting 1 runs, first five are: 21c6f56a:46221f12-6f01-4877-8453-c69d992257e1
[2023-07-31 21:54:12Z] Execution of experiment failed, update experiment status

JobException: Exception : 
 {
    "error": {
        "code": "UserError",
        "message": "Pipeline has failed child jobs. Failed nodes: /notify_job. For more details and logs, please go to the job detail page and check the child jobs.",
        "message_format": "Pipeline has failed child jobs. {0}",
        "message_parameters": {},
        "reference_code": "PipelineHasStepJobFailed",
        "details": []
    },
    "environment": "northeurope",
    "location": "northeurope",
    "time": "2023-07-31T21:54:12.482718Z",
    "component_name": ""
} 